In [1]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.stats import boxcox
from scipy import sparse
from sklearn.model_selection import StratifiedKFold
from itertools import product
from sklearn import preprocessing

from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss

from subprocess import check_output
from scipy.stats import pearsonr

from os import listdir
from os.path import isfile, join
from scipy.stats import rankdata
import zipfile

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
ensemble_path = 'ensemble/oof'

In [5]:
def merge_all_result(path,key,df):
    file_list = []
    file_path_list=[]
    try:
        for file in listdir(ensemble_path):

            if file.find('.csv')==-1:
                continue

            print(file)    
            file_list.append(file)
            file_path = join(ensemble_path,file)
            file_path_list.append(file_path)
            temp = pd.read_csv(file_path)
            #predict_list.append(temp[LABELS].values)
            if df.empty == True:
                print("empty")
                df = df.append(temp)
            else:
                print("in")
                df = df.merge(temp,on=key)
    except BaseException as e:
        print("exception")
        print(e)
    finally:
        return df

In [6]:
predict_list = []
data = pd.DataFrame()
data = merge_all_result(ensemble_path,'card_id',data)

hyeonwoo_test_oof.csv
empty
hyeonwoo_test_oof_auc.csv
in
hyeonwoo_test_oof_cat_auc.csv
in
hyeonwoo_test_oof_xgb_auc.csv
in
hyeonwoo_train_oof.csv
in
hyeonwoo_train_oof_auc.csv
in
hyeonwoo_train_oof_cat_auc.csv
in
hyeonwoo_train_oof_xgb_auc.csv
in
test_oof_augmented_2500_auc_0.9057314966569757_logloss__function log_loss at 0x7f9ab3285bf8__v1.csv
in
test_oof_augmented_2500_cv_auc_0.9053867482566532_logloss_0.043834212933958186_v2.csv
empty
exception
Plan shapes are not aligned


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [44]:
if 'target' in data.columns:
    devide_val = data.shape[1]-1
    data['target'] = data['target_x'].sum(axis=1) + data['target_x'].sum(axis=1) + data['target']
    data['target'] = data['target']/devide_val
    del data['target_x'], data['target_y']
else:
    devide_val = data.shape[1]-1
    data['target'] = data['target_x'].sum(axis=1) + data['target_x'].sum(axis=1)
    data['target'] = data['target']/devide_val
    del data['target_x'], data['target_y']

In [50]:
data.loc[data['card_id']=='C_ID_944c62886f','target'] = -33.21928095

In [51]:
data.shape

(123623, 2)

In [52]:
data.loc[data['target']<-30].shape

(39, 2)

In [53]:
data.to_csv('under_650_result_blending.csv',index=False)

In [13]:
def calculate_correlation(base_df,target):
    
    source = base_df.copy()
    source = source.merge(target,on='card_id')
    corr_df = source.corr()
    
    corr1 = corr_df.ix['target_x']['target_y']
    del corr_df,source
    return corr1

In [14]:
def is_same_listing_id(source,target):
    return 1 if np.sum(source['card_id'] - target['card_id']) == 0 else 0

In [15]:
ensemble_base_file = '20190225_174621_submission_blend_blend.csv'

In [16]:
file_list = []
file_path_list=[]
base_df = pd.read_csv(join(ensemble_path,ensemble_base_file))
corr_dict = dict()
try:
    for file in listdir(ensemble_path):
        
        if file.find('.csv')==-1:
            continue
            
        print(file)    
        file_list.append(file)
        file_path = join(ensemble_path,file)
        file_path_list.append(file_path)
        print(file_path)
        temp = pd.read_csv(file_path)

        #if is_same_listing_id(base_df,temp) == 0:    
        #    print('error')
        #    continue

        c = calculate_correlation(base_df,temp)
        corr_dict[file]=c
        print(file,c)
        print("")
        #if df.empty == True:
        #    print("empty")
        #    df = df.append(temp)
        #else:
        #    print("in")
        #    df = df.merge(temp,on='listing_id')
except:
    print("exception")

20190214_154826_submission_blend_blend.csv
ensemble/test6\20190214_154826_submission_blend_blend.csv


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


20190214_154826_submission_blend_blend.csv 0.9933586346809608

20190215_235228_submission_blend_blend.csv
ensemble/test6\20190215_235228_submission_blend_blend.csv
20190215_235228_submission_blend_blend.csv 0.992778502303612

20190217_090058_submission_blend_blend.csv
ensemble/test6\20190217_090058_submission_blend_blend.csv
20190217_090058_submission_blend_blend.csv 0.9940342181845591

20190218_185933_submission_blend_blend.csv
ensemble/test6\20190218_185933_submission_blend_blend.csv
20190218_185933_submission_blend_blend.csv 0.9936965340661271

20190221_155851_submission_blend_blend.csv
ensemble/test6\20190221_155851_submission_blend_blend.csv
20190221_155851_submission_blend_blend.csv 0.9939540708975336

20190222_143608_submission_blend_blend.csv
ensemble/test6\20190222_143608_submission_blend_blend.csv
20190222_143608_submission_blend_blend.csv 0.9939191313377468

20190222_144911_submission_blend_blend.csv
ensemble/test6\20190222_144911_submission_blend_blend.csv
20190222_144911_s